In [33]:
# Instalar a biblioteca mlxtend
!pip install mlxtend
!pip install pandas

2101.14s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


Defaulting to user installation because normal site-packages is not writeable


2107.27s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


Defaulting to user installation because normal site-packages is not writeable


# Leitura do Dataset e Algoritmo Apriori
Neste notebook, vamos carregar o dataset `base_dados.csv` localizado na pasta `db` e aplicar o algoritmo Apriori para extrair regras de associação.

In [1]:
# Importar bibliotecas necessárias
import pandas as pd
from mlxtend.frequent_patterns import apriori, association_rules

# Carregar o dataset
dataset_path = 'db/base_dados.csv'
df = pd.read_csv(dataset_path)
df = df.replace({'sim': 1, 'não': 0})
df = df.drop(columns=['No'])  

# Exibir as primeiras linhas do dataset
df.head()
df


/tmp/ipykernel_36620/2079678922.py:8: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.replace({'sim': 1, 'não': 0})


,Leite,Café,Cerveja,Pão,Manteiga,Arroz,Feijão
0,0,1,0,1,1,0,0
1,1,0,1,1,1,0,0
2,0,1,0,1,1,0,0
3,1,1,0,1,1,0,0
4,0,0,1,0,0,0,0
5,0,0,0,0,1,0,0
6,0,0,0,1,0,0,0
7,0,0,0,0,0,0,1
8,0,0,0,0,0,1,1
9,0,0,0,0,0,1,0


In [3]:
from mlxtend.frequent_patterns import apriori

class Apriori:
    """
    Classe Apriori para executar o algoritmo Apriori e filtrar os resultados.
    """

    def __init__(self, df, threshold=0.5, transform_bol=False):
        """
        Construtor da classe Apriori.

        :param pandas.DataFrame df: Dataset transacional (valores 1 ou 0).
        :param float threshold: Suporte mínimo para o algoritmo Apriori.
        :param bool transform_bol: Indica se os dados devem ser transformados para valores booleanos.
        """
        self.df = df.astype(bool)  # Converte os dados para o tipo bool
        self.threshold = threshold

    def _apriori(self, use_colnames=False, max_len=None, count=True):
        """
        Executa o algoritmo Apriori usando a biblioteca mlxtend.

        :param bool use_colnames: Indica se os nomes das colunas devem ser usados no resultado.
        :param int max_len: Tamanho máximo dos conjuntos de itens gerados.
        :param bool count: Indica se a contagem do tamanho dos conjuntos deve ser incluída.
        :return: DataFrame com os resultados do Apriori.
        :rtype: pandas.DataFrame
        """
        apriori_df = apriori(
            self.df,
            min_support=self.threshold,
            use_colnames=use_colnames,
            max_len=max_len
        )
        if count:
            apriori_df['length'] = apriori_df['itemsets'].apply(lambda x: len(x))
        return apriori_df

    def run(self, use_colnames=False, count=True):
        """
        Executa o algoritmo Apriori para gerar pares de itens.

        :param bool use_colnames: Indica se os nomes das colunas devem ser usados no resultado.
        :param bool count: Indica se a contagem do tamanho dos conjuntos deve ser incluída.
        :return: DataFrame com os resultados do Apriori.
        :rtype: pandas.DataFrame
        """
        return self._apriori(use_colnames=use_colnames, max_len=2, count=count)

    def filter(self, apriori_df, threshold):
        """
        Filtra os resultados do Apriori com base no suporte.

        :param pandas.DataFrame apriori_df: DataFrame com os resultados do Apriori.
        :param float threshold: Suporte mínimo desejado.
        :return: DataFrame filtrado.
        :rtype: pandas.DataFrame
        """
        return apriori_df[apriori_df['support'] >= threshold]

    def filter_pairs(self, apriori_df, threshold):
        """
        Filtra os resultados do Apriori para incluir apenas pares de itens.

        :param pandas.DataFrame apriori_df: DataFrame com os resultados do Apriori.
        :param float threshold: Suporte mínimo desejado.
        :return: DataFrame filtrado com pares de itens.
        :rtype: pandas.DataFrame
        """
        return apriori_df[(apriori_df['length'] == 2) & (apriori_df['support'] >= threshold)]


# Transformar os dados para o tipo bool
df = df.astype(bool)

# Executar o algoritmo Apriori para pares de itens
apriori_runner = Apriori(df, threshold=0.2, transform_bol=True)  # Reduzir o threshold, se necessário
apriori_df = apriori_runner.run(use_colnames=True, count=True)

# Filtrar pares de itens com suporte maior que 0.2
filtered_pairs = apriori_runner.filter_pairs(apriori_df, threshold=0.2)

print("Pares de itens com suporte maior que 0.2:")
print(filtered_pairs)

Pares de itens com suporte maior que 0.2:
    support           itemsets  length
7       0.2       (Pão, Leite)       2
8       0.2  (Manteiga, Leite)       2
9       0.3        (Pão, Café)       2
10      0.3   (Manteiga, Café)       2
11      0.4    (Manteiga, Pão)       2
